In [1]:
from hera.shared import global_config
from hera.auth import ArgoCLITokenGenerator

global_config.host = "https://localhost:2746"
global_config.token = ArgoCLITokenGenerator
global_config.verify_ssl = False

## Hera - compiles workflow template manifest (i.e. define `Flow`)

WorkFlowTemplates crucially allow the definition of parameters as input variables to the entire workflow *without specifying a value*. You can also specify the default value, but it is possible to omit in cases where a sensible value just doesnt apply.

Submitting these WorkFlow templates doesnt execute anything on the ArgoWorkflow engine, but rather adds a template that future `Workflow`s can be derived from, together with configurable workflow input parameters.

This allows for the following entity mapping between bettmensch.ai pipelines and ArgoWorkflow resources:

| bettmensch.ai | ArgoWorkflow      |
| --------------|-------------------|
| `Flow`        | `WorkflowTemplate`|
| `Run`         | `Workflow`        |

In [54]:
from hera.workflows import WorkflowTemplate, DAG, script, Parameter, models as m

@script(outputs=[Parameter(name="sum",value_from=m.ValueFrom(path="./sum.txt"))])
def add(a: float, b: float):
    with open('./sum.txt','w') as output:
        output.write(str(a + b))
        
@script(outputs=[Parameter(name="difference",value_from=m.ValueFrom(path="./difference.txt"))])
def subtract(a: float, b: float):
    with open('./difference.txt','w') as output:
        output.write(str(a - b))
        
@script(outputs=[Parameter(name="product",value_from=m.ValueFrom(path="./product.txt"))])
def multiply(a: float, b: float):
    with open('./product.txt','w') as output:
        output.write(str(a * b))

dag_name = "2ab-minus-bcd"

with WorkflowTemplate(
    generate_name="2ab-minus-bcd-", 
    entrypoint=dag_name,
    namespace="argo",
    arguments=[Parameter(name="a"),Parameter(name="b"),Parameter(name="c"),Parameter(name="d")],
    # the Workflow referencing this template inherits the `spec` level `workflow_metadata` field from this WorkflowTemplate, so
    # when submitting via .create(), the workflow metadata does get merged into the Workflow CRD manifest before being applied to the K8s cluster
    workflow_metadata={
        'annotations':{'annotation_key_a':'annotation_value_a'},
        'labels':{'label_key_a':'label_value_a'}
    },
    # the Workflow referencing this template inherits the `spec` level `pod_metadata` field from this WorkflowTemplate, so
    # when submitting via .create(), the workflow metadata does get merged into the Pod manifest(s) before being applied to the K8s cluster.
    # for whatever reason, the (apparently?) pod level manifests shown on the ArgoWorkflow server dashboard do not show this
    pod_metadata={ 
        'annotations':{'annotation_key_b':'annotation_value_b'},
        'labels':{'label_key_b':'label_value_b'}
    }) as wt:
    
    wt_a, wt_b, wt_c, wt_d = wt.get_parameter('a').value,wt.get_parameter('b').value,wt.get_parameter('c').value,wt.get_parameter('d').value 
    
    with DAG(name=dag_name):
        double_a = multiply(
            name="multiply-2-and-a",arguments=[
                Parameter(name="a",value=2), 
                Parameter(name="b",value=wt_a)
            ]
        )
        bc = multiply(
            name="multiply-b-and-c",arguments=[
                Parameter(name="a",value=wt_b),
                Parameter(name="b",value=wt_c)
            ]
        )
        double_ab = multiply(
            name="multiply-2a-and-b",arguments=[
                Parameter(name="a",value=double_a.get_parameter('product').value),
                Parameter(name="b",value=wt_b)
            ]
        )
        bcd = multiply(
            name="multiply-bc-and-d",arguments=[
                Parameter(name="a",value=bc.get_parameter('product').value),
                Parameter(name="b",value=wt_d)
            ]
        )
        double_ab_minus_bcd = subtract(
            name="subtract-2ab-and-bcd",arguments=[
                Parameter(name="a",value=double_ab.get_parameter('product').value),
                Parameter(name="b",value=bcd.get_parameter('product').value)
            ],    
        )
        
        double_a >> double_ab
        bc >> bcd
        [double_ab, bcd] >> double_ab_minus_bcd       

In [55]:
import yaml

with open('hera-template-3.2.yaml','w') as pipeline_file:
    yaml.dump(wt.to_yaml(),pipeline_file)

# Hera - submit Workflow template (i.e. submit `Flow`)

In [56]:
response = wt.create()
wt_name = response.metadata.name

c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


# Hera - generate workflow from template (i.e. configure `Run`)

In [57]:
from hera.workflows import Workflow
from hera.workflows.models import WorkflowTemplateRef

# using the workflow template reference
wt_ref = WorkflowTemplateRef(name=wt_name)

w1 = Workflow(
    generate_name=f"{wt_name}-expect-20-",
    workflow_template_ref=wt_ref,
    namespace="argo",
    arguments=[
        Parameter(name="a",value='1'),
        Parameter(name="b",value='2'),
        Parameter(name="c",value='3'),
        Parameter(name="d",value='4')
    ]
) # -20

w2 = Workflow(
    generate_name=f"{wt_name}-expect-0-",
    workflow_template_ref=wt_ref,
    namespace="argo",
    arguments=[
        Parameter(name="a",value='1.5'),
        Parameter(name="b",value='2'),
        Parameter(name="c",value='1.5'),
        Parameter(name="d",value='2')
    ]
) # 0

# Hera - submit workflow (i.e. execute `Run`)

In [58]:
import yaml

with open('hera-pipeline-3.2.1-from-ref.yaml','w') as pipeline_file:
    yaml.dump(w1.to_yaml(),pipeline_file)

with open('hera-pipeline-3.2.2-from-ref.yaml','w') as pipeline_file:
    yaml.dump(w2.to_yaml(),pipeline_file)
    
response_w1 = w1.create()
response_w2 = w2.create()

print(response_w1)
print(response_w2)

c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


api_version=None kind=None metadata=ObjectMeta(annotations=None, cluster_name=None, creation_timestamp=Time(__root__=datetime.datetime(2024, 5, 3, 11, 18, 11, tzinfo=datetime.timezone.utc)), deletion_grace_period_seconds=None, deletion_timestamp=None, finalizers=None, generate_name='2ab-minus-bcd-7vt8p-expect-20-', generation=1, labels={'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, managed_fields=[ManagedFieldsEntry(api_version='argoproj.io/v1alpha1', fields_type='FieldsV1', fields_v1=FieldsV1(), manager='argo', operation='Update', subresource=None, time=Time(__root__=datetime.datetime(2024, 5, 3, 11, 18, 11, tzinfo=datetime.timezone.utc)))], name='2ab-minus-bcd-7vt8p-expect-20-g7lg8', namespace='argo', owner_references=None, resource_version='166973', self_link=None, uid='e7195650-d09e-4cdc-af58-fee171a1f176') spec=WorkflowSpec(active_deadline_seconds=None, affinity=None, archive_logs=None, arguments=Arguments(artifacts=None, parameters=[Parameter(default